In [1]:
state_name = 'WB'
state_full_name = 'West Bengal'
warehouse_id_1 = 8
warehouse_id_2 = 10

In [45]:
import math
import json
import time
import random
import shutil
import gspread
import numpy as np
import pandas as pd
from google.colab import userdata
from datetime import datetime as dt
from google.oauth2 import service_account
from googleapiclient.discovery import build
from gspread_dataframe import set_with_dataframe
from googleapiclient.http import MediaFileUpload

In [3]:
# ✅ 1. Load credentials from Colab Secrets
secret_string = userdata.get("gspread_credentials")
info_dict = json.loads(secret_string)

SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = service_account.Credentials.from_service_account_info(info_dict, scopes=SCOPES)

# ✅ 2. Authorize gspread
gc = gspread.authorize(creds)

# ✅ 3. Open the Google Sheet (make sure it's shared with service account)
workbook = gc.open(f"Automated Store Ordering {state_name} PY - 2")

# ✅ 4. Used here for Drive API
drive_service = build("drive", "v3", credentials=creds)

In [4]:
# Exclusion store from 60 DOH Capping

exclusion_stores = ['AM Barasat KOL', 'AM Sonar Mandir HM KOL']

In [5]:

def repeat_stores(input_list, n):
    # Convert the list to a numpy array
    array = np.array(input_list)

    # Using numpy's repeat function to repeat each element
    repeated_array = np.repeat(array, n, 0)

    return repeated_array

In [6]:
def mround(number, multiple):
  return round(number / multiple + 0.0001) * multiple

In [7]:
def makeOrder(curre_qty, ars, tres_doh, tar_doh, modi_mbq, mrq, ct_name, st_name):
    # Equation 1: MBQ + DOH with 1
    if ct_name == 'Sankranti':
        if (curre_qty / ars) > tres_doh and curre_qty >= modi_mbq:
            return 0
        else:
            return mround(
                math.ceil(
                    max(((tar_doh - (curre_qty / ars)) * ars) - 0.00001,
                        (max(modi_mbq, mrq) - curre_qty)) / 2
                ) * 2,
                mrq
            )

    # Equation 2: MBQ Only
    elif ct_name in ['clip_strip', 'bcd', 'super_deals', 'wow_week', 'mt', 'mt_ad_hoc', 'visibility', 'gift', 'focus_skus', 'asm']:
        mbq_only = max(math.ceil(((modi_mbq - curre_qty) / mrq) - 0.00001) * mrq, 0)

        # Apply safeguard if DOH >= 60
        if (mbq_only + curre_qty) / ars >= 60 and st_name not in exclusion_stores and ct_name not in ['mt', 'visibility', 'gift']:
            if curre_qty == 0:
                return 2
            elif (curre_qty / ars) >= 60:
                return 0
            else:
                return math.ceil(max(((60 - (curre_qty / ars)) * ars) - 0.00001, 1))
        else:
            return mbq_only

    # Equation 3: MBQ + DOH with 1.5
    else:
        if (curre_qty / ars) > tres_doh and curre_qty >= (modi_mbq / 1.5):
            return 0
        else:
            final_qty = mround(
                math.ceil(
                    max(((tar_doh - (curre_qty / ars)) * ars) - 0.00001,
                        (max(modi_mbq, mrq) - curre_qty)) / 2
                ) * 2,
                mrq
            )

            # Apply safeguard if DOH >= 60
            if (final_qty + curre_qty) / ars >= 60 and st_name not in exclusion_stores and ct_name not in ['loose']:
                if curre_qty == 0:
                    return 2
                elif (curre_qty / ars) >= 60:
                    return 0
                else:
                    return math.ceil(max(((60 - (curre_qty / ars)) * ars) - 0.00001, 1))
            else:
                return final_qty

In [8]:
def cal_thres(beat,tar,cat):
  if cat == 'fmcg_a' or cat == 'fmcg_b' or cat == 'fmcg_c' or cat == 'staples_a' or cat == 'staples_b' or cat == 'staples_c' or cat == 'gm_a' or cat == 'gm_b' or cat == 'gm_c' or cat == 'loose' or cat == 'jit' or cat == 'jit-k' or cat == 'summer' or cat == 'btl' or cat == 'npi' or cat == 'clip_strip' or cat == 'bcd' or cat == 'super_deals' or cat == 'wow_week' or cat == 'mt' or cat == 'mt_ad_hoc' or cat == 'visibility' or cat == 'gift' or cat == 'focus_skus' or cat == 'asm' or cat == 'allocation' or cat == 'depletion':
    return beat + tar
  # elif cat in ('non-core'):
  #   return round(tar/2)
  else:
    return beat + 3

def cal_tar(thres,tar,cat):
  # if cat == 'fmcg_head' or cat == 'staples' or cat == 'staples_jit' or cat == 'loose':
  if cat == 'fmcg_a' or cat == 'fmcg_b' or cat == 'fmcg_c' or cat == 'staples_a' or cat == 'staples_b' or cat == 'staples_c' or cat == 'gm_a' or cat == 'gm_b' or cat == 'gm_c' or cat == 'loose' or cat == 'jit' or cat == 'jit-k' or cat == 'summer' or cat == 'btl' or cat == 'npi' or cat == 'clip_strip' or cat == 'bcd' or cat == 'super_deals' or cat == 'wow_week' or cat == 'mt' or cat == 'mt_ad_hoc' or cat == 'visibility' or cat == 'gift' or cat == 'focus_skus' or cat == 'asm' or cat == 'allocation' or cat == 'depletion':
    return thres
  # elif cat in ('non-core'):
  #   return tar
  else:
    return thres + tar

In [9]:
def cal_mbq(multiplier, mbq, cat):
  if cat == 'mt_ad_hoc' or cat == 'visibility' or cat == 'loose':
    return mbq
  elif cat == 'gift':
    return round(multiplier*mbq,1)
  else:
    return mround((multiplier*mbq),2)

In [10]:
def dfCreator(cat_df,cat_name):
  # creating store x item list

  if cat_name == 'mt_ad_hoc' or cat_name == 'visibility' or cat_name == 'gift':
    joined_df1 = cat_df
  else:
    st_details = cat_df.loc[:,["tez_id","samaan_id","store_name","beat_gap","multiplier"]]
    st_details = st_details[st_details['tez_id'] != 0]
    it_details = cat_df.loc[:,["item_code","target_doh","mbq"]]
    it_details = it_details[it_details['item_code'] != 0]
    st_half = pd.DataFrame(repeat_stores(st_details,it_details.shape[0]),columns=st_details.columns)
    it_half = pd.DataFrame(np.tile(np.array(it_details),(st_details.shape[0],1)),columns=it_details.columns)
    joined_df1 = st_half.join(it_half)


  joined_df1['type'] = cat_name
  joined_df1['multiplier'] = joined_df1['multiplier'].apply(lambda x: np.around(x,1))
  joined_df1['mod_mbq'] = joined_df1.apply(lambda x: cal_mbq(x['multiplier'],x['mbq'],cat_name),axis=1)
  joined_df1 = pd.merge(joined_df1,mrq_df2.loc[:,['item_code','display_name','min_replenish_qty']],on=['item_code'],how='left')
  print('merge 1')
  joined_df1 = pd.merge(joined_df1,inv_df2,on=['store_name','item_code'],how='left')
  print('merge 2')
  transit_df2.rename(columns={'name':'store_name'},inplace=True)
  joined_df1 = pd.merge(joined_df1,transit_df2.loc[:,['store_name','item_code','process_qty']],on=['store_name','item_code'],how='left')
  print('merge 3')
  # cart_df.rename(columns={'store_id':'samaan_id'},inplace=True)
  # joined_df1 = pd.merge(joined_df1,cart_df2,on=['samaan_id','item_code'],how='left')
  print('merge 4')
  joined_df1 = pd.merge(joined_df1,avl_df2,left_on=['tez_id','item_code'],right_on=['store_id','item_code'],how='left')
  print('merge 5')
  joined_df1.drop(columns=['store_id'],inplace=True)
  joined_df1['curr_qty'] = joined_df1['curr_qty'].fillna(0)
  joined_df1['process_qty'] = joined_df1['process_qty'].fillna(0)
  # joined_df1['order_qty'] = joined_df1['order_qty'].fillna(0)
  joined_df1['count_avl_days'] = joined_df1['count_avl_days'].fillna(0.01)
  joined_df1['count_avl_days'] = joined_df1['count_avl_days'].apply(lambda x: 15 if x < 15 else x)
  joined_df1['last_30daysales'] = joined_df1['last_30daysales'].fillna(0.01)
  joined_df1['last_30daysales'] = joined_df1['last_30daysales'].apply(lambda x: x if x > 0 else 0.01)
  # joined_df1 = joined_df1.assign(all_qty=lambda row: row['curr_qty'] + row['process_qty'] + row['order_qty']) # Calculating All qty
  joined_df1 = joined_df1.assign(all_qty=lambda row: row['curr_qty'] + row['process_qty']) # Calculating All qty
  joined_df1 = joined_df1.assign(curr_doh=lambda row: row['curr_qty']/(row['last_30daysales']/row['count_avl_days'])) # Calculating curr DOH
  # joined_df1.drop(columns=['curr_qty','process_qty','order_qty'],inplace=True) # Removing Extra Columns
  joined_df1.drop(columns=['curr_qty','process_qty'],inplace=True) # Removing Extra Columns
  joined_df1 = joined_df1.assign(ARS=lambda x: x['last_30daysales']/x['count_avl_days'])
  joined_df1['ARS'] = joined_df1['ARS'].apply(lambda x: max(x,0.01))

  # Calculating Threshold and Target DOH

  joined_df1['threshold_doh'] = joined_df1.apply(lambda x: cal_thres(x['beat_gap'],x['target_doh'],cat_name),axis=1)
  joined_df1['target_doh'] = joined_df1.apply(lambda x: cal_tar(x['threshold_doh'],x['target_doh'],cat_name) ,axis=1)
  joined_df1 = pd.merge(joined_df1,wh_df2.loc[:,['sku_code','wh_qty']],left_on=['item_code'],right_on=['sku_code'],how='left')
  print('merge 6')
  joined_df1['wh_qty'] = joined_df1['wh_qty'].fillna(0)

  # print(joined_df1)
  joined_df1['order'] = joined_df1.apply(lambda x: makeOrder(x['all_qty'],x['ARS'],x['threshold_doh'],x['target_doh'],x['mod_mbq'],x['min_replenish_qty'],cat_name,x['store_name']),axis=1)

  # return joined_df1[joined_df1['order'] != 0].loc[:,['item_code','store_name','order','type']]
  # return joined_df1[joined_df1['order'] != 0]
  return joined_df1




In [11]:
# Ordering sheet selections by the user
gids = {
        ##### Main Assortment #####

        'fmcg_a':2140842897,
        'fmcg_b':1459512030,
        'fmcg_c':1024514954,
        'staples_a':686195830,
        'staples_b':2098196922,
        'staples_c':965871084,
        'gm_a':1645717487,
        'gm_b':941148715,
        'gm_c':314829402,
        'loose':53967786,
        'jit':197437796,
        'jit-k':1553249338,

        ##### Temp Assortment #####

        'clip_strip':55763387,
        'bcd':279157190,
        'visibility':1994951969,
        'gift':1080084985,
        'npi':788140546,
        'summer':168005917,
        'asm':2075309652,
        'focus_skus':960399022,

      # 'btl':0,
      # 'super_deals':360075548,
      # 'wow_week':2111882564,
      # 'mt':440498050,
      # 'mt_ad_hoc':913226990,
      # 'allocation':341955460,
      # 'depletion':1974209973
        }

In [12]:
wh_gids = {
        ##### Main Assortment #####

        'fmcg_a':2140842897,
        'fmcg_b':1459512030,
        'fmcg_c':1024514954,
        'staples_a':686195830,
        'staples_b':2098196922,
        'staples_c':965871084,
        'gm_a':1645717487,
        'gm_b':941148715,
        'gm_c':314829402,
        'loose':53967786,
        'jit':197437796,
        'jit-k':1553249338,

        ##### Temp Assortment #####

        'clip_strip':55763387,
        'bcd':279157190,
        'npi':788140546,
        'summer':168005917,
        'asm':2075309652,
        'focus_skus':960399022,

      # 'btl':0,
      # 'super_deals':360075548,
      # 'wow_week':2111882564,
      # 'mt':440498050,
      # 'mt_ad_hoc':913226990,
      # 'visibility':1994951969,
      # 'gift':1080084985,
      # 'allocation':341955460,
      # 'depletion':1974209973
        }

# DataBase Inputs

In [13]:
######### DB Imports #########

sheet_id = '1f-Ty8gMlzi1fvm8wpNdX7GFjqHk9pytI5eGtvTpvc2Q'
gids_import = {
    "Avl Days": "0",
    "Inventory of foco": "1706153079",
    "MRQ and Price": "1884639423",
    "Transit": "514474300",
    "Warehouse Inventory": "1325392378"
}

### Inventory of foco ###
try:
    inv_df2 = pd.read_csv('input inventory of foco.csv')
    print('input taken: inventory of foco')
except:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gids_import['Inventory of foco']}"
    inv_df2 = pd.read_csv(url)
    print('Loaded from Google Sheet: Inventory of foco')

### Avl Days ###
try:
    avl_df2 = pd.read_csv('input available days of foco.csv')
    print('input taken: avl days')
except:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gids_import['Avl Days']}"
    avl_df2 = pd.read_csv(url)
    print('Loaded from Google Sheet: Avl Days')

### Transit ###
try:
    transit_df2 = pd.read_csv('input in-transit items of foco.csv')
    print('input taken: transit')
except:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gids_import['Transit']}"
    transit_df2 = pd.read_csv(url)
    print('Loaded from Google Sheet: Transit')

### MRQ and Price ###
try:
    mrq_df2 = pd.read_csv('input mrq and price.csv')
    print('input taken: MRQ and Price')
except:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gids_import['MRQ and Price']}"
    mrq_df2 = pd.read_csv(url)
    print('Loaded from Google Sheet: MRQ and Price')

### Warehouse Inventory ###
try:
    wh_df2 = pd.read_csv('input wh inventory.csv')
    print('input taken: warehouse inventory')
except:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gids_import['Warehouse Inventory']}"
    wh_df2 = pd.read_csv(url)
    print('Loaded from Google Sheet: Warehouse Inventory')

Loaded from Google Sheet: Inventory of foco
Loaded from Google Sheet: Avl Days
Loaded from Google Sheet: Transit
Loaded from Google Sheet: MRQ and Price
Loaded from Google Sheet: Warehouse Inventory


In [14]:
# Converting all numeric columns to Float
l1 = [
        inv_df2,
        avl_df2,
        # cart_df2,
        transit_df2,
        mrq_df2,
        wh_df2
     ]

for ele in l1:
  for col in ele:
    try:
      ele[col] = ele[col].astype(np.float32)
    except:
      print(f'{col} is a string column')

# cart_df2 = cart_df2.groupby(['item_code','samaan_id'],as_index=False).agg({'order_qty':'sum'})
transit_df2 = transit_df2.groupby(['item_code','name'],as_index=False).agg({'process_qty':'sum'})


store_name is a string column
display_name is a string column
name is a string column
status is a string column
display_name is a string column
master_category is a string column
category is a string column
sub_category is a string column
leaf_category is a string column
product_type is a string column
display_name is a string column


In [15]:
# SQL Query result Output

# inv_df2.to_csv("inventory of foco.csv",index=False)
# avl_df2.to_csv("available days of foco.csv",index=False)
# cart_df2.to_csv("in-cart items of foco.csv",index=False)
# transit_df2.to_csv("in-transit items of foco.csv",index=False)
# wh_df2.to_csv("wh stock of foco.csv",index=False)


# Remaining Ordering Sheet

In [27]:
# Store Ordering
# Scanning Ordering files from Google Sheet (Automated Store Ordering - Python)
# Safe retry function
def safe_get_values(ws, rng, max_retries=5):
    for i in range(max_retries):
        try:
            return ws.get_values(rng)
        except Exception as e:
            if '429' in str(e):
                wait = 2 ** i + random.uniform(0, 1)
                print(f"⚠️  Quota exceeded. Retrying in {wait:.2f} seconds...")
                time.sleep(wait)
            else:
                raise
    raise Exception("❌ Max retries reached for get_values")

# 🧾 Store Ordering
raw_df = {}
for name in gids.keys():
    worksheet = workbook.get_worksheet_by_id(gids[name])
    time.sleep(1)  # Pause to avoid rate limit
    print(f"📥 Fetching: {name}")

    rows = safe_get_values(worksheet, 'A2:H')
    df_items = pd.DataFrame.from_records(rows)
    df_items.columns = ['tez_id','samaan_id','store_name','beat_gap','multiplier','item_code','target_doh','mbq']

    df_items.loc[:, ['tez_id','samaan_id','beat_gap','multiplier','item_code','target_doh','mbq']] = \
        df_items.loc[:, ['tez_id','samaan_id','beat_gap','multiplier','item_code','target_doh','mbq']].replace('', np.nan)

    df_items['item_code'] = df_items['item_code'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)

    df_items = df_items.astype({
        'tez_id': np.float16,
        'samaan_id': np.float16,
        'beat_gap': np.float16,
        'multiplier': np.float16,
        'item_code': np.float32,
        'target_doh': np.float16,
        'mbq': np.float16
    })

    raw_df[name] = df_items

📥 Fetching: fmcg_a
📥 Fetching: fmcg_b
📥 Fetching: fmcg_c
📥 Fetching: staples_a
📥 Fetching: staples_b
📥 Fetching: staples_c
📥 Fetching: gm_a
📥 Fetching: gm_b
📥 Fetching: gm_c
📥 Fetching: loose
📥 Fetching: jit
📥 Fetching: jit-k
📥 Fetching: clip_strip
📥 Fetching: bcd
📥 Fetching: visibility
📥 Fetching: gift
📥 Fetching: npi
📥 Fetching: summer
📥 Fetching: asm
📥 Fetching: focus_skus


In [25]:
# Warehouse Ordering

# 🧠 Retry wrapper for quota-safe get_values
def safe_get_values(ws, rng, max_retries=5):
    for i in range(max_retries):
        try:
            return ws.get_values(rng)
        except Exception as e:
            if '429' in str(e):
                wait = 2 ** i + random.uniform(0, 1)
                print(f"⏳ Quota exceeded. Retrying in {wait:.2f} seconds...")
                time.sleep(wait)
            else:
                raise
    raise Exception("🚨 Max retries reached for get_values.")

# 🧾 Main warehouse data loop
wh_raw_df = {}
for name in wh_gids.keys():
    worksheet = workbook.get_worksheet_by_id(wh_gids[name])
    time.sleep(1)  # ✅ Reduce rapid calls

    print(f"📥 Fetching: {name}")
    rows = safe_get_values(worksheet, 'A2:H')

    df_items = pd.DataFrame.from_records(rows)
    df_items.columns = ['tez_id','samaan_id','store_name','beat_gap','multiplier','item_code','target_doh','mbq']

    df_items.loc[:, ['tez_id','samaan_id','beat_gap','multiplier','item_code','target_doh','mbq']] = \
        df_items.loc[:, ['tez_id','samaan_id','beat_gap','multiplier','item_code','target_doh','mbq']].replace('', np.nan)

    df_items['item_code'] = df_items['item_code'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)

    df_items = df_items.astype({
        'tez_id': np.float16,
        'samaan_id': np.float16,
        'beat_gap': np.float16,
        'multiplier': np.float16,
        'item_code': np.float32,
        'target_doh': np.float16,
        'mbq': np.float16
    })

    wh_raw_df[name] = df_items

📥 Fetching: fmcg_a
📥 Fetching: fmcg_b
📥 Fetching: fmcg_c
📥 Fetching: staples_a
📥 Fetching: staples_b
📥 Fetching: staples_c
📥 Fetching: gm_a
📥 Fetching: gm_b
📥 Fetching: gm_c
📥 Fetching: loose
📥 Fetching: jit
📥 Fetching: jit-k
📥 Fetching: clip_strip
📥 Fetching: bcd
📥 Fetching: npi
📥 Fetching: summer
📥 Fetching: asm
📥 Fetching: focus_skus


In [28]:
# GENERATING ORDER Store Order

# Making calulations on the basis of given params and Generating Orders

names = raw_df.keys()
cooked_dfs = {}

for ele in names:
  print(ele)
  cooked_dfs[ele] = dfCreator(raw_df[ele],ele)

print(f'Orders are successfully generated!')

fmcg_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
fmcg_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
fmcg_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
loose
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
jit
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
jit-k
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
clip_strip
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
bcd
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
visibility
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gift
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
npi
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
summer
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
asm
merge 1


In [29]:
# GENERATING ORDER Warehouse

wh_names = wh_raw_df.keys()
wh_cooked_dfs = {}

for ele in wh_names:
  print(ele)
  wh_cooked_dfs[ele] = dfCreator(wh_raw_df[ele],ele)

print(f'wh_Orders are successfully generated!')

fmcg_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
fmcg_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
fmcg_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
staples_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_a
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_b
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
gm_c
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
loose
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
jit
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
jit-k
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
clip_strip
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
bcd
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
npi
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
summer
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
asm
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
focus_skus
merge 1
merge 2
merge 3
merge 4
merge 5
merge 6
wh_Orders are

In [30]:
# Stacking all columns, compiling complete ordering sheet

collated_df = pd.concat(cooked_dfs.values()).reset_index(drop=True)

In [31]:
# Warehouse order collate

wh_collated_df = pd.concat(wh_cooked_dfs.values()).reset_index(drop=True)

In [32]:
wh_collated_df[wh_collated_df['type'] == 'winter']

,tez_id,samaan_id,store_name,beat_gap,multiplier,item_code,target_doh,mbq,type,mod_mbq,...,min_replenish_qty,last_30daysales,count_avl_days,all_qty,curr_doh,ARS,threshold_doh,sku_code,wh_qty,order


In [33]:
# Creating OD File, (i.e. Item x Store x Max Qty)

grouped_df = collated_df.groupby(['item_code','store_name'],as_index=False).agg({'order':'max'})
for_details = grouped_df.copy()
grouped_df = pd.merge(grouped_df,collated_df.loc[:,['item_code','store_name','order','type']],on=['item_code','store_name','order'],how='left')

# OD File
grouped_df = grouped_df.drop_duplicates(subset=['item_code','store_name','order'])
grouped_df = grouped_df[grouped_df['order'] != 0]

# Detailed OD File
details_df = pd.merge(for_details,collated_df,on=['item_code','store_name','order'],how='left')
details_df = details_df.drop_duplicates(subset=['item_code','store_name','order'])
details_df = details_df[details_df['order'] != 0]


In [34]:
# Final Warehouse Order

wh_grouped_df = wh_collated_df.groupby(['item_code','store_name'],as_index=False).agg({'order':'max'})
# for_details = wh_grouped_df.copy()

# Adding Type column
wh_grouped_df = pd.merge(wh_grouped_df,wh_collated_df.loc[:,['item_code','store_name','order','type']],on=['item_code','store_name','order'],how='left')

# Dropping Duplicates
wh_grouped_df = wh_grouped_df.drop_duplicates(subset=['item_code','store_name','order'])

# Removing zeros
# wh_grouped_df = wh_grouped_df[wh_grouped_df['order'] != 0]

### Exclusion Part

In [35]:
exclusionList = workbook.get_worksheet_by_id(156791444)
rows = exclusionList.get_values('A2:C')
ex_items = pd.DataFrame.from_records(rows)
ex_items = ex_items[[1, 2]]
ex_items.columns = ['store_name','item_code']
ex_items.loc[:,['store_name','item_code']].replace(to_replace='',value=np.nan,inplace=True)
ex_items['item_code'] = ex_items['item_code'].apply(lambda x: x.replace(',',''))
ex_items = ex_items.astype({'item_code':np.float32})

# Create a boolean mask for rows to keep (not in ex_items)
ex_items = ~grouped_df[['store_name', 'item_code']].apply(tuple, axis=1).isin(ex_items[['store_name', 'item_code']].apply(tuple, axis=1))

# Apply the mask
grouped_df = grouped_df[ex_items]

### Output Part

In [36]:
# Compiled OD file to CSV
grouped_df.to_csv('od_output.csv',index=False)

# Compiled Detail OD file to CSV
# details_df.to_csv('od_details_output.csv',index=False)

# Compiled complete ordering sheet to CSV
collated_df.to_csv('complete_output.csv',index=False)

In [50]:
# ✅ File already saved earlier
local_path = '/content/complete_output.csv'

# ✅ Just change the filename in Drive, not locally
new_filename = f"complete_details_{str(dt.today().date())}.csv"

# ✅ Upload to Drive
folder_id = "1ZY8tCyVrFLz36rVw0kx4eMObf781OS2d"  # Replace with your Drive folder ID

file_metadata = {
    'name': new_filename,
    'parents': [folder_id]
}

media = MediaFileUpload(local_path, mimetype='text/csv')

file = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

print("complete_output uploaded to drive")

complete_output uploaded to drive


In [38]:
# Open the Google Sheet and select the specified worksheet
specified_worksheet = workbook.worksheet("Store Order")  # Change to your target sheet name

# Clear the specific range in the specified worksheet
specified_worksheet.batch_clear(["A2:D"])  # Adjust the range as needed

# Write the raw collated DataFrame to the specified worksheet
set_with_dataframe(specified_worksheet, grouped_df, include_index=False, include_column_header=True, row=2, col=1)

print("✅ Successfully cleared the specified range and updated with store order.")

✅ Successfully cleared the specified range and updated with store order.


In [51]:
# Pasting Store Demand in Automated Store Ordering CG sheet

output_sheet = workbook.get_worksheet_by_id(1789363742)
cell_list = output_sheet.get_values('A2:B')

tempi_df = pd.DataFrame.from_records(cell_list)

size = tempi_df.shape

wh_grouped_df.to_csv('warehouse_output.csv',index=False)
warehouse_demand_df = wh_grouped_df.groupby(['item_code'],as_index=False).agg({'order':'sum'})

# Deleting Previous Output

check = np.full(size,'')
output_sheet.update(pd.DataFrame(check).values.tolist(),"A2")

# Pasting new Output

warehouse_demand_df = wh_grouped_df.groupby(['item_code'],as_index=False).agg({'order':'sum'})
output_sheet.update(warehouse_demand_df.values.tolist(),"A2")

{'spreadsheetId': '1SjnzZybBzCUZozqrEIhF2PKnm-SYHTCaALQYjm5oVvY',
 'updatedRange': "'Warehouse Order'!A2:B4780",
 'updatedRows': 4779,
 'updatedColumns': 2,
 'updatedCells': 9558}

In [40]:
# warehouse_demand_df.to_csv('warehouse_demand.csv',index=False)